In [4]:
from sklearn.metrics import cohen_kappa_score
k=cohen_kappa_score
import pandas as pd
import numpy as np
import glob
import os
import csv
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [475]:
import nltk
from gensim.models import Word2Vec
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

# checking which excel files has more positive values.

In [476]:
df_topic_lockdowns=pd.read_csv("/Users/abhishekshastry/Documents/Illinois_tech/Subjects/Fall-2021/NLP/Assignments/Assignment2/Solution/Primary_dataset/nyt_topic_lockdowns_final_primary_dataset.csv")

In [477]:
# count_lockdowns=0
# for x in df_topic_lockdowns.final_label.values:
#        if x==True:
#         count_lockdowns=count_lockdowns+1
# print('Positive lockdown values', count_lockdowns)

In [478]:
print(df_topic_lockdowns['final_label'].value_counts())

False    711
True     102
Name: final_label, dtype: int64


In [479]:
df_topic_masking_distancing=pd.read_csv("/Users/abhishekshastry/Documents/Illinois_tech/Subjects/Fall-2021/NLP/Assignments/Assignment2/Solution/Primary_dataset/nyt_topic_masking_and_distancing_final_primary_dataset.csv")

In [480]:
# count_masking=0
# for x in df_topic_masking_distancing.final_label.values:
#        if x==True:
#         count_masking=count_masking+1
# print('Positive masking_distancing values', count_masking)

In [481]:
print(df_topic_masking_distancing['final_label'].value_counts())

False    586
True     227
Name: final_label, dtype: int64


In [482]:
df_topic_vaccination=pd.read_csv("/Users/abhishekshastry/Documents/Illinois_tech/Subjects/Fall-2021/NLP/Assignments/Assignment2/Solution/Primary_dataset/nyt_topic_vaccination_final_primary_dataset.csv")

In [483]:
# count_vaccination=0
# for x in df_topic_vaccination.final_label.values:
#        if x==True:
#         count_vaccination=count_vaccination+1
# print('Positive count_vaccination values', count_vaccination)
# print(df_topic_vaccination['final_label'].value_counts())

In [484]:
print(df_topic_vaccination['final_label'].value_counts())

True     496
False    317
Name: final_label, dtype: int64


# I will be considering vaccination as a topic as it has most positive values.

# Primary raw data set

In [485]:
df_topic_vaccination.data=df_topic_vaccination['text']
df_topic_vaccination.target=df_topic_vaccination['final_label']
df_topic_vaccination.data.shape

<ipython-input-485-e9bff2b00ed9>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_topic_vaccination.data=df_topic_vaccination['text']
<ipython-input-485-e9bff2b00ed9>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_topic_vaccination.target=df_topic_vaccination['final_label']


(813,)

In [486]:
df_topic_vaccination['final_label'].shape

(813,)

# using gausian naive bayes

In [529]:
count_vect = CountVectorizer()
df_X_train_primary = count_vect.fit_transform(df_topic_vaccination.data)
df_X_train_primary.shape

(813, 6274)

In [530]:
tfidf_transformer = TfidfTransformer()
df_X_train_tfidf = tfidf_transformer.fit_transform(df_X_train_primary).toarray()
df_X_train_tfidf.shape

(813, 6274)

In [533]:
X_train_primary, X_test_primary, y_train_primary, y_test_primary = train_test_split(df_X_train_tfidf, df_topic_vaccination.target, test_size = 0.3, random_state = 0)
model = GaussianNB()
gausian_model_primary=model.fit(X_train_primary,y_train_primary)
y_pred_primary_gausian = gausian_model_primary.predict(X_test_primary)
print(accuracy_score(y_test_primary, y_pred_primary_gausian))
print(f1_score(y_test_primary, y_pred_primary_gausian))
print(precision_score(y_test_primary,y_pred_primary_gausian))

0.6762295081967213
0.7641791044776118
0.6918918918918919


In [534]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(y_test_primary, y_pred_primary_gausian,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(y_test_primary,y_pred_primary_gausian)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.63      0.39      0.48        94
       FALSE       0.69      0.85      0.76       150

    accuracy                           0.68       244
   macro avg       0.66      0.62      0.62       244
weighted avg       0.67      0.68      0.66       244

[[ 37  57]
 [ 22 128]]


# using gausian bayes on set of primary baseline model accuarcy is 68%

# using Multinomial naive bayes

In [535]:
nb_model_primary = MultinomialNB(alpha=1.0).fit(X_train_primary, y_train_primary)
y_pred_Mnb_primary = nb_model_primary.predict(X_test_primary)
print(accuracy_score(y_test_primary, y_pred_Mnb_primary))
print(f1_score(y_test_primary, y_pred_Mnb_primary))
print(precision_score(y_test_primary, y_pred_Mnb_primary))

0.6147540983606558
0.7614213197969544
0.6147540983606558


In [536]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(y_test_primary, y_pred_Mnb_primary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(y_test_primary,y_pred_Mnb_primary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.00      0.00      0.00        94
       FALSE       0.61      1.00      0.76       150

    accuracy                           0.61       244
   macro avg       0.31      0.50      0.38       244
weighted avg       0.38      0.61      0.47       244

[[  0  94]
 [  0 150]]


/Users/abhishekshastry/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abhishekshastry/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abhishekshastry/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

# using multinomial bayes on set of primary baseline model accuarcy is 61%

# using random forest classifier

In [537]:
classifier_rf = RandomForestClassifier(n_estimators=100)
classifier_rf.fit(X_train_primary, y_train_primary)
y_pred_rf_primary=classifier_rf.predict(X_test_primary)
print(accuracy_score(y_test_primary,y_pred_rf_primary))
print(f1_score(y_test_primary, y_pred_rf_primary))
print(precision_score(y_test_primary,y_pred_rf_primary))

0.8360655737704918
0.8780487804878049
0.8089887640449438


In [538]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(y_test_primary, y_pred_rf_primary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(y_test_primary,y_pred_rf_primary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.91      0.64      0.75        94
       FALSE       0.81      0.96      0.88       150

    accuracy                           0.84       244
   macro avg       0.86      0.80      0.81       244
weighted avg       0.85      0.84      0.83       244

[[ 60  34]
 [  6 144]]


# using random forest on set of primary baseline model accuarcy is 84%

# Testing on secondary raw data set

In [539]:
df_topic_vaccination_secondary=pd.read_csv("/Users/abhishekshastry/Documents/Illinois_tech/Subjects/Fall-2021/NLP/Assignments/Assignment2/Solution/secondary_dataset/change_org_topic_vaccination_final_secondary_dataset.csv")

In [540]:
df_topic_vaccination_secondary.data=df_topic_vaccination_secondary['text']
df_topic_vaccination_secondary.target=df_topic_vaccination_secondary['final_label']

<ipython-input-540-47f7843e63ea>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_topic_vaccination_secondary.data=df_topic_vaccination_secondary['text']
<ipython-input-540-47f7843e63ea>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_topic_vaccination_secondary.target=df_topic_vaccination_secondary['final_label']


# Using the already fitted Multinomial classifier 

In [543]:
df_X_test_secondary = count_vect.transform(df_topic_vaccination_secondary['text'])
print(df_X_test_secondary.shape)
df_X__test_tfidf_secondary = tfidf_transformer.transform(df_X_count_secondary)
print(df_X__test_tfidf_secondary.shape)
# using trained nb classifer
y_pred_Mnb_secondary= nb_model_primary.predict(df_X__test_tfidf_secondary)
print(accuracy_score(df_topic_vaccination_secondary.target, y_pred_Mnb_secondary))
print(f1_score(df_topic_vaccination_secondary.target, y_pred_Mnb_secondary))
print(precision_score(df_topic_vaccination_secondary.target, y_pred_Mnb_secondary))

(1500, 6274)
(1500, 6274)
0.037333333333333336
0.03604806408544726
0.018354860639021073


In [544]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(df_topic_vaccination_secondary.target, y_pred_Mnb_secondary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(df_topic_vaccination_secondary.target, y_pred_Mnb_secondary)
print(cm)

              precision    recall  f1-score   support

        TRUE       1.00      0.02      0.04      1473
       FALSE       0.02      1.00      0.04        27

    accuracy                           0.04      1500
   macro avg       0.51      0.51      0.04      1500
weighted avg       0.98      0.04      0.04      1500

[[  29 1444]
 [   0   27]]


# using the secondary test data test with multinomial classifier we get an accuracy of 4 percent.

# Using the already fitted Random Forest Classifier

In [545]:
df_X_rf_secondary = count_vect.transform(df_topic_vaccination_secondary['text'])
print(df_X_rf_secondary.shape)
df_X__rf_tfidf_secondary = tfidf_transformer.transform(df_X_rf_secondary)
print(df_X__rf_tfidf_secondary.shape)
# using trained nb classifer
y_pred_rf_secondary= classifier_rf.predict(df_X__rf_tfidf_secondary)
print(accuracy_score(df_topic_vaccination_secondary.target, y_pred_rf_secondary))
print(f1_score(df_topic_vaccination_secondary.target, y_pred_rf_secondary))
print(precision_score(df_topic_vaccination_secondary.target, y_pred_rf_secondary))

(1500, 6274)
(1500, 6274)
0.9826666666666667
0.2777777777777778
0.5555555555555556


In [546]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(df_topic_vaccination_secondary.target, y_pred_rf_secondary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(df_topic_vaccination_secondary.target, y_pred_rf_secondary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.99      1.00      0.99      1473
       FALSE       0.56      0.19      0.28        27

    accuracy                           0.98      1500
   macro avg       0.77      0.59      0.63      1500
weighted avg       0.98      0.98      0.98      1500

[[1469    4]
 [  22    5]]


# using the secondary test data test with random classifier we get an accuracy of 98  percent.

# Feature engineering

# Here feature extraction involves making use of regular expression, stop words, stemming, tokenization, n grams, word frequencies

# Feature engineering on primary data set

In [547]:
def clean_text(df):
    topic_vaccination = list()
    lines = df_topic_vaccination["text"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        #lemmatizer = WordNetLemmatizer()
        words = [PS.stem(w) for w in words if not w in stop_words]
        #words = [lemmatizer.lemmatize(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        topic_vaccination.append(words)
        
    return topic_vaccination

clean_topic_vaccination_data_primary= clean_text(df_topic_vaccination.data)
print(len(clean_topic_vaccination_data_primary))
#display(df_topic_vaccination["text"][0])

813


# sticking to stemming instead of lemmentisation as lemmentisation was treating vaccination, vaccinated, vaccines all uniquely.  stop words are used and where in 'not' is  discarded. 

# extracting word frequencies

In [183]:
lines = df_topic_vaccination["text"].values.tolist()
lst_words = ["vaccin", "protect", "safe",'sick','motiv']
## count
lst_grams = [len(word.split(" ")) for word in lst_words]
vectorizer = feature_extraction.text.CountVectorizer(vocabulary=lst_words, ngram_range=(min(lst_grams),max(lst_grams)))
dict = {'Feature_Engineered_Text': clean_topic_vaccination_data_primary} 
fe_df = pd.DataFrame(dict)
#fe_df
dwf = pd.DataFrame(vectorizer.fit_transform(fe_df["Feature_Engineered_Text"]).todense(), columns=lst_words)
# ## add the new features as columns
df_wf = pd.concat([fe_df, dwf.set_index(fe_df.index),df_topic_vaccination.target],axis=1)

x=df_wf.iloc[:,1]+df_wf.iloc[:,2]+df_wf.iloc[:,3]+df_wf.iloc[:,4]+df_wf.iloc[:,5]
df_wf.iloc[:,6:7]
df_wf.iloc[:,1:6]

,vaccin,protect,safe,sick,motiv
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
808,0,0,0,0,0
809,2,0,0,0,0
810,2,0,0,0,0
811,0,0,0,0,0


In [32]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Using Multinomial classifer on feature engineered primary data set without word frequency and n grams

In [424]:
fe_X_train_primary,fe_X_test_primary, fe_y_train_primary, fe_y_test_primary=train_test_split(clean_topic_vaccination_data_primary, df_topic_vaccination.target, test_size = 0.3, random_state = 0)

In [425]:
fe_count_vect = CountVectorizer()
feature_extraction_df_X_train_primary = fe_count_vect.fit_transform(fe_X_train_primary)
print(fe_count_vect.get_feature_names)
feature_extraction_df_X_train_primary.shape

<bound method CountVectorizer.get_feature_names of CountVectorizer()>


(569, 3514)

In [426]:
fe_tfidf_transformer = TfidfTransformer()
fe_df_X_train_tfidf_primary = fe_tfidf_transformer.fit_transform(feature_extraction_df_X_train_primary).toarray()
fe_df_X_train_tfidf_primary.shape
fe_df_X_train_tfidf_primary.shape

(569, 3514)

In [427]:
fe_nb_model_primary = MultinomialNB(alpha=1.0).fit(fe_df_X_train_tfidf_primary, fe_y_train_primary)
fe_X_test_primary_cv=fe_count_vect.transform(fe_X_test_primary)
fe_X_test_primary_tf=fe_tfidf_transformer.transform(fe_X_test_primary_cv)
fe_y_pred_Mnb_primary = fe_nb_model_primary.predict(fe_X_test_primary_tf)
print(accuracy_score(fe_y_test_primary, fe_y_pred_Mnb_primary))
print(f1_score(fe_y_test_primary, fe_y_pred_Mnb_primary))
print(precision_score(fe_y_test_primary, fe_y_pred_Mnb_primary))

0.6557377049180327
0.7801047120418848
0.6422413793103449


In [428]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(fe_y_test_primary, fe_y_pred_Mnb_primary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(fe_y_test_primary,fe_y_pred_Mnb_primary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.92      0.12      0.21        94
       FALSE       0.64      0.99      0.78       150

    accuracy                           0.66       244
   macro avg       0.78      0.56      0.49       244
weighted avg       0.75      0.66      0.56       244

[[ 11  83]
 [  1 149]]


# Using Multinomial classifer on feature engineered word frequency primary data set

In [178]:
wf_X_train_primary,wf_X_test_primary, wf_y_train_primary, wf_y_test_primary = train_test_split(df_wf.iloc[:,1:6],df_wf.iloc[:,6], test_size = 0.3, random_state =0)

In [405]:
#taking transformed tf of primary train set(count vectorised text data)
fe_df_X_train_tfidf_primary

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [404]:
#word frequency columns of primary training data set 
wf_train_primary=wf_X_train_primary.to_numpy()
wf_train_primary

array([[6, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [2, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [322]:
# concatinating the primary data
wf_df_X_train_primary=pd.DataFrame(np.concatenate((fe_df_X_train_tfidf_primary,wf_train_primary),axis=1))
wf_df_X_train_primary.shape

(569, 3519)

In [366]:
#taking transformed tf of primary test set(count vectorised text data)
fe_X_test_primary_tf_array=fe_X_test_primary_tf.toarray()
fe_X_test_primary_tf_array.shape

(244, 3514)

In [453]:
##word frequency columns of primary test data set 
wf_X_test_primary_array=wf_X_test_primary.to_numpy()
wf_X_test_primary_array.shape

(244, 5)

In [452]:
wf_df_X_test_primary=pd.DataFrame(np.concatenate((fe_X_test_primary_tf_array,wf_X_test_primary_array),axis=1))
#wf_df_X_test_primary=np.concatenate((v,wf_test_primary), axis=0)
wf_df_X_test_primary.shape

(244, 3519)

In [383]:
wf_nb_model_primary = MultinomialNB(alpha=1.0).fit(wf_df_X_train_primary,wf_y_train_primary)
wf_y_pred_Mnb_primary = wf_nb_model_primary.predict(wf_df_X_test_primary)
print(accuracy_score(wf_y_test_primary, wf_y_pred_Mnb_primary))
print(f1_score(wf_y_test_primary, wf_y_pred_Mnb_primary))
print(precision_score(wf_y_test_primary,wf_y_pred_Mnb_primary))

0.7827868852459017
0.848137535816619
0.7437185929648241


# Therefore by applying word frequency and n grams, 78 percent of accuracy was achievable which increased from before which was 61 percent.

# Using Random forest classifier on featured engineered primary data set.

In [548]:
fe_classifier_rf = RandomForestClassifier(n_estimators=100)
fe_classifier_rf.fit(fe_df_X_train_tfidf_primary,fe_y_train_primary)
fe_y_pred_rf_primary=fe_classifier_rf.predict(fe_X_test_primary_tf)
print(accuracy_score(fe_y_test_primary,fe_y_pred_rf_primary))
print(f1_score(fe_y_test_primary, fe_y_pred_rf_primary))
print(precision_score(fe_y_test_primary,fe_y_pred_rf_primary))

0.9098360655737705
0.9276315789473684
0.9155844155844156


In [549]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(fe_y_test_primary, fe_y_pred_rf_primary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(fe_y_test_primary,fe_y_pred_rf_primary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.90      0.86      0.88        94
       FALSE       0.92      0.94      0.93       150

    accuracy                           0.91       244
   macro avg       0.91      0.90      0.90       244
weighted avg       0.91      0.91      0.91       244

[[ 81  13]
 [  9 141]]


# With random forest classifier, we get 87 percentage of accurage on primary test(0.3) data set

# Using Random forest classifer on feature engineered word frequency primary data set

In [550]:
fe_classifier_rf_wf = RandomForestClassifier(n_estimators=100)
fe_classifier_rf_wf.fit(wf_df_X_train_primary,wf_y_train_primary)
fe_y_pred_rf_primary_wf=fe_classifier_rf_wf.predict(wf_df_X_test_primary)
print(accuracy_score(fe_y_test_primary,fe_y_pred_rf_primary_wf))
print(f1_score(fe_y_test_primary, fe_y_pred_rf_primary_wf))
print(precision_score(fe_y_test_primary,fe_y_pred_rf_primary_wf))

0.9180327868852459
0.9324324324324323
0.9452054794520548


In [551]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(fe_y_test_primary, fe_y_pred_rf_primary_wf,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(fe_y_test_primary,fe_y_pred_rf_primary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.88      0.91      0.90        94
       FALSE       0.95      0.92      0.93       150

    accuracy                           0.92       244
   macro avg       0.91      0.92      0.91       244
weighted avg       0.92      0.92      0.92       244

[[ 81  13]
 [  9 141]]


# after applying above feature extraction on the primary data set for random classification model, the accuarcy increased from 91 percent to 92 percent.

# Feature engineering on secondary test data set

In [394]:
def clean_text(df):
    topic_vaccination_secondary = list()
    lines = df.values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        #lemmatizer = WordNetLemmatizer()
        words = [PS.stem(w) for w in words if not w in stop_words]
        #words = [lemmatizer.lemmatize(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        topic_vaccination_secondary.append(words)
    return topic_vaccination_secondary

clean_topic_vaccination_data_secondary= clean_text(df_topic_vaccination_secondary.data)
clean_topic_vaccination_data_secondary[0:20]

['award',
 'cancel school year sol',
 '',
 'protect nurs',
 'senseless imprison mom first time offend nonviol crime',
 'demand immedi releas incarcer knox counti jail',
 'petit recal desmond elliot lago hous assembl suruler constitu',
 'texa uil spring sportscompetit complet summer',
 'priorit vaccin florida teacher',
 'save plum island coffe roaster',
 'suspend rent mortgag canada',
 'ask citi winstonsalem support instreet dine downtown restur',
 'onlin class close physic bmcc campu due outbreak',
 'appello per la pace etiopia',
 'stop guinea pig vaccin test africa covid corona viru',
 'rowan univers univers pass system',
 'allow athlet therapist treat patient',
 'allow not covid relat evict',
 'freeneko navajo counti jail',
 'özel okul']

# Using the already fitted Multinomial classifier's model on featured secondary dataset..

In [417]:
# using the word frequency method for secondary entire test data set.

In [418]:
lines = df_topic_vaccination_secondary["text"].values.tolist()
lst_words = ["vaccin", "protect", "safe",'sick','motiv']
## count
lst_grams = [len(word.split(" ")) for word in lst_words]
vectorizer = feature_extraction.text.CountVectorizer(vocabulary=lst_words, ngram_range=(min(lst_grams),max(lst_grams)))
dict_secondary = {'Feature_Engineered_Text_secondary': clean_topic_vaccination_data_secondary} 
fe_df = pd.DataFrame(dict_secondary)
dwf_second = pd.DataFrame(vectorizer.fit_transform(fe_df["Feature_Engineered_Text_secondary"]).todense(), columns=lst_words)
# ## add the new features as columns
df_wf_second = pd.concat([fe_df,dwf_second.set_index(fe_df.index),df_topic_vaccination_secondary.target],axis=1)
df_wf_second
# x=df_wf.iloc[:,1]+df_wf_second.iloc[:,2]+df_wf.iloc[:,3]+df_wf.iloc[:,4]+df_wf.iloc[:,5]
# df_wf.iloc[:,6:7]
# df_wf.iloc[:,1:6]

,Feature_Engineered_Text_secondary,vaccin,protect,safe,sick,motiv,final_label
0,award,0,0,0,0,0,False
1,cancel school year sol,0,0,0,0,0,False
2,,0,0,0,0,0,False
3,protect nurs,0,1,0,0,0,False
4,senseless imprison mom first time offend nonvi...,0,0,0,0,0,False
...,...,...,...,...,...,...,...
1495,null void premier leagu,0,0,0,0,0,False
1496,free public transport nh staff accompani pay rise,0,0,0,0,0,False
1497,pm pleas give immedi covid insur cover nh staff,0,0,0,0,0,False
1498,say osha propos perman rule,0,0,0,0,0,False


In [422]:
df_wf_second.iloc[:,0:1]

,Feature_Engineered_Text_secondary
0,award
1,cancel school year sol
2,
3,protect nurs
4,senseless imprison mom first time offend nonvi...
...,...
1495,null void premier leagu
1496,free public transport nh staff accompani pay rise
1497,pm pleas give immedi covid insur cover nh staff
1498,say osha propos perman rule


In [445]:
#count vectorising the text data of secondary entire data set
fe_count_vect_second=fe_count_vect.transform(clean_topic_vaccination_data_secondary)
fe_tfidf_transformer_second=fe_tfidf_transformer.transform(fe_count_vect_second)
fe_tfidf_transformer_second_array=fe_tfidf_transformer_second.toarray()
fe_tfidf_transformer_second_array.shape

(1500, 3514)

In [ ]:
#taking the word frequency into consideration.. 

In [442]:
wf__second_array=df_wf_second.iloc[:,1:6].to_numpy()
wf__second_array.shape

(1500, 5)

In [450]:
# concatinating the secondary data for Multinaives bayes prediction..
wf_df_X_test_secondary=pd.DataFrame(np.concatenate((fe_tfidf_transformer_second_array,wf__second_array),axis=1))
wf_df_X_test_secondary.shape

(1500, 3519)

In [455]:
#using trained nb classifer of featured engineer..
fe_y_pred_Mnb_secondary= wf_nb_model_primary.predict(wf_df_X_test_secondary)
print(accuracy_score(df_topic_vaccination_secondary.target, fe_y_pred_Mnb_secondary))
print(f1_score(df_topic_vaccination_secondary.target, fe_y_pred_Mnb_secondary))
print(precision_score(df_topic_vaccination_secondary.target,fe_y_pred_Mnb_secondary))

0.306
0.04931506849315069
0.025280898876404494


In [552]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(df_topic_vaccination_secondary.target, fe_y_pred_Mnb_secondary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(df_topic_vaccination_secondary.target, fe_y_pred_Mnb_secondary)
print(cm)

              precision    recall  f1-score   support

        TRUE       1.00      0.29      0.45      1473
       FALSE       0.03      1.00      0.05        27

    accuracy                           0.31      1500
   macro avg       0.51      0.65      0.25      1500
weighted avg       0.98      0.31      0.45      1500

[[ 432 1041]
 [   0   27]]


# The accuracy for the model is 31 percent.which has increased from 4 percent of secondary data set without feature extraction.This confirms the increase in the accuracy after feature extraction for multinomial bayes secondary test data set 

# Using the already fitted Random Forest Classifier model on featured secondary data set..

In [459]:
# using trained random forest classifier....
fe_y_pred_rf_secondary= fe_classifier_rf_wf.predict(wf_df_X_test_secondary)
print(accuracy_score(df_topic_vaccination_secondary.target, fe_y_pred_rf_secondary))
print(f1_score(df_topic_vaccination_secondary.target, fe_y_pred_rf_secondary))
print(precision_score(df_topic_vaccination_secondary.target, fe_y_pred_rf_secondary))

0.9933333333333333
0.782608695652174
0.9473684210526315


In [204]:
df_topic_vaccination_target_names=["TRUE","FALSE"]
print(metrics.classification_report(df_topic_vaccination_secondary.target, fe_y_pred_rf_secondary,target_names=df_topic_vaccination_target_names))
cm = metrics.confusion_matrix(df_topic_vaccination_secondary.target, fe_y_pred_rf_secondary)
print(cm)

              precision    recall  f1-score   support

        TRUE       0.99      0.99      0.99      1473
       FALSE       0.58      0.67      0.62        27

    accuracy                           0.99      1500
   macro avg       0.79      0.83      0.81      1500
weighted avg       0.99      0.99      0.99      1500

[[1460   13]
 [   9   18]]


# The accuracy for the model is 99 percent.which has increased from 98 percent of secondary data set without feature extraction.This confirms the increase in the accuracy after feature extraction for random forest secondary test data set 